In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
import time
from utils import writeJson
import os

In [11]:
def refuseCookies(driver):
    try: 
        cookies = driver.find_element(By.ID, "onetrust-reject-all-handler")
        cookies.click()       
    except:
        pass

def showMore(driver):
    show_more_matches = True
    refuseCookies(driver)
    while show_more_matches:
        try:
            button = driver.find_elements(By.CLASS_NAME, "event__more" )
            if len(button) == 1:
                button = button[0]
            else:
                print(len(button))
                button = None
            if button == None:
                show_more_matches = False
            else:
                #print(button.get_attribute('href'), button.text)
                actions = ActionChains(driver)
                actions.move_to_element(button).perform()
                time.sleep(1)
                #driver.execute_script("arguments[0].scrollIntoView(true);", button)
                actions.click(button).perform()
        
        except StaleElementReferenceException:
            show_more_matches = False

        except NoSuchElementException:
            show_more_matches = False

def getComments(url_commento: str): 
    driver_match = webdriver.Chrome()
    driver_match.get(url_commento)
    refuseCookies(driver_match)
    teams = [x.text for x in driver_match.find_elements(By.CLASS_NAME, 'participant__participantName')]
    tags = driver_match.find_elements(By.TAG_NAME, 'a')
    day = [x.text for x in tags if 'GIORNATA' in x.text]
    if(len(day) > 0):
        day = day[0].split(' ')[-1]
    else:
        day = 'xx'
        
    if len(teams) == 4:
        home, away = teams[0], teams[2]
    else:
        home, away = teams[0], teams[1]
    comments_button = [x for x in driver_match.find_elements(By.CLASS_NAME, '_tab_i2rza_4') if x.text == 'COMMENTO'][0]
    comments_button.click()
    time.sleep(1)
    comments = [x.text for x in driver_match.find_elements(By.CLASS_NAME, '_general_1vhh5_4')]
    driver_match.quit()
    return day, home, away, comments

        

In [5]:
def mainExtractionComments(url: str, out_path: str):
    driver = webdriver.Chrome()
    driver.get(url)
    showMore(driver)
    matches = driver.find_elements(By.CLASS_NAME, 'eventRowLink')
    matches_urls = [x.get_property('href') for x in matches]
    driver.quit()
    for match in matches_urls:
        day, home, away, comment_def =getComments(match)
        file_out = os.path.join(out_path, f'{day}_{home}_{away}.json')
        writeJson(comment_def, file_out)

In [12]:
seasons = ['2023-2024', '2022-2023', '2021-2022', '2020-2021', '2019-2020']
leagues = {'italia': ['Serie A'],'inghilterra': ['Premier League'], 'germania': ['Bundesliga'], 'francia': ['Ligue 1'], 'spagna': ['LaLiga'], 'portogallo': ['Liga Portugal'], 'europa':['Champions League', 'Europa League'],'olanda': ['Eredivisie']}
dataset_path = "Dataset//Direttait//"
for s in seasons:
    for nat, league in leagues.items():
        for l in league:
            if(nat != 'italy' and s != '2023-2024'):
                l_url = l.replace(' ', '-').lower()
                url = f'https://www.diretta.it/calcio/{nat}/{l_url}-{s}/risultati/'
                print(f'start {l} {s}')
                league_path = os.path.join(dataset_path, l)
                if not os.path.exists(league_path):
                    os.makedirs(league_path)
                season_league_path = os.path.join(league_path, s)
                if not os.path.exists(season_league_path):
                    os.makedirs(season_league_path)
                mainExtractionComments(url, season_league_path)
                print(f'end {l} {s}')
            

start Serie A 2022-2023
